In [80]:
import pandas as pd
import numpy as np

In [81]:
data = pd.read_csv('sancionesCnbv.csv')

In [82]:
data.columns = [x.lower() for x in data.columns]

In [83]:
data.columns = data.columns.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')

In [84]:
data.columns= data.columns.str.replace(' ' ,"_")

In [85]:
data.columns = data.columns.str.strip('_')

In [86]:
data.columns

Index(['id_sancion', 'tipo_de_persona', 'infractor', 'tipo_de_sancion',
       'sub_sector', 'monto', 'entidad_asociada', 'sector_de_entidad_asociada',
       'conducta_sancionada', 'normatividad_infringida',
       'descripcion_de_la_conducta', 'fecha_de_infraccion',
       'fecha_de_imposicion', 'pagada', 'grave', 'reincidente',
       'resolucion_firme', 'susceptible_de_impugnacion', 'mes_de_publicacion',
       'aclaraciones'],
      dtype='object')

In [87]:
data = data.rename(columns={'fecha_de_infraccion':'anio_infraccion'})

In [88]:
data['monto'] = data['monto'].astype('str') 
data ["monto"] = data ["monto"].str.extract(r'([\d].*\.\d.)')
data ["monto"] = data ["monto"].str.replace(',', '')
data [['monto']] = data[['monto']].fillna(0).astype(float)

In [89]:
data ['anio_infraccion']=data['anio_infraccion'].astype('string').str.rstrip(".0")

In [90]:
data['anio_infraccion']= pd.to_datetime(data['anio_infraccion'],errors='coerce')

In [91]:
data['fecha_de_imposicion']= pd.to_datetime(data['fecha_de_imposicion'],format='%d/%m/%Y',errors='coerce')



In [92]:
data['anio_imposicion'] = data['fecha_de_imposicion'].dt.strftime('%Y')

In [93]:
data['anio_infraccion'] = data['anio_infraccion'].dt.strftime('%Y')

In [94]:
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

infractor                         1
tipo_de_sancion                  14
sub_sector                      362
entidad_asociada              16139
sector_de_entidad_asociada    16139
descripcion_de_la_conducta       56
anio_infraccion                1193
fecha_de_imposicion               6
aclaraciones                   5727
anio_imposicion                   6
dtype: int64

In [95]:
drop_cols = list(null_cols[null_cols > 1500].index)


In [96]:
data = data.drop(drop_cols, axis=1)

In [97]:
data=data.drop(data[data['infractor'].isnull()==True].index, axis=0)

In [98]:
data=data.drop(data[data['anio_infraccion'].isnull()==True].index, axis=0)
data=data.drop(data[data['fecha_de_imposicion'].isnull()==True].index, axis=0)

In [99]:
data = data.drop('mes_de_publicacion', axis=1)

In [100]:
def acentos_lower(x):
    return x.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8').str.lower()
data['infractor'] = data[['infractor']].apply(acentos_lower)
data['tipo_de_persona'] = data[['tipo_de_persona']].apply(acentos_lower)
data['conducta_sancionada'] = data[['conducta_sancionada']].apply(acentos_lower)
data['tipo_de_sancion'] = data[['tipo_de_sancion']].apply(acentos_lower)
data['descripcion_de_la_conducta'] = data[['descripcion_de_la_conducta']].apply(acentos_lower)

In [101]:

data ["infractor"] = data ["infractor"].str.extract(r'((\w.*(?= s\.a))|(\w.*(?= a\.c))|(\w.*(?=, a\.g))|(\w.*(?= s\.c))|(\w.*(?= sapi de c))|\w.*)')


In [102]:
data['infractor']= data['infractor'].str.rstrip('. ').str.rstrip(',').str.rstrip(' ') 

In [103]:
data=data.drop(data[data['infractor'].isnull()==True].index, axis=0)

In [104]:

data['descripcion_de_la_conducta'] = data['descripcion_de_la_conducta'].fillna('sin informacion')

In [105]:
#data['descripcion_de_la_conducta'][data['descripcion_de_la_conducta'].str.contains('lavado|identificacion de clientes|alto riesgo|operaciones inusuales|operaciones relevantes|operaciones preocupantes', regex=True)].unique().tolist()

In [107]:
condition1= [ data['conducta_sancionada'].str.contains('lavado|identificacion de clientes|alto riesgo|operaciones inusuales|operaciones relevantes|operaciones preocupantes', regex=True),\
            data['descripcion_de_la_conducta'].str.contains('lavado|identificacion de clientes|alto riesgo|operaciones inusuales|operaciones relevantes|operaciones preocupantes', regex=True)]
choices1= ['Si','Si']

data['lavado'] = np.select(condition1,choices1, default= 'No')

#data['lavado'] = np.where(data[['conducta_sancionada','descripcion_de_la_conducta']].str.contains('lavado|identificacion de clientes|alto riesgo|operaciones inusuales|operaciones relevantes|operaciones preocupantes', regex=True), 'Si' , 'No')


In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14947 entries, 0 to 16146
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id_sancion                  14947 non-null  int64         
 1   tipo_de_persona             14947 non-null  object        
 2   infractor                   14947 non-null  object        
 3   tipo_de_sancion             14935 non-null  object        
 4   sub_sector                  14651 non-null  object        
 5   monto                       14947 non-null  float64       
 6   conducta_sancionada         14947 non-null  object        
 7   normatividad_infringida     14947 non-null  object        
 8   descripcion_de_la_conducta  14947 non-null  object        
 9   anio_infraccion             14947 non-null  object        
 10  fecha_de_imposicion         14947 non-null  datetime64[ns]
 11  pagada                      14947 non-null  object    

In [109]:
data

,id_sancion,tipo_de_persona,infractor,tipo_de_sancion,sub_sector,monto,conducta_sancionada,normatividad_infringida,descripcion_de_la_conducta,anio_infraccion,fecha_de_imposicion,pagada,grave,reincidente,resolucion_firme,susceptible_de_impugnacion,anio_imposicion,lavado
0,48073,fisica,leon cabrera flores,amonestacion,NaN,0.0,lmv - el sistema automatizado para la recepcio...,"LMV - Art. Regulador: 180 Art. Sanción: 392, f...","el sistema automatizado para la recepcion, reg...",2015,2017-11-30,No,No,No,No,Si,2017,No
1,48074,fisica,jaime antonio gonzalez remis,amonestacion,NaN,0.0,lmv - el sistema automatizado para la recepcio...,"LMV - Art. Regulador: 180 Art. Sanción: 392, f...","el sistema automatizado para la recepcion, reg...",2015,2017-11-30,No,No,No,No,Si,2017,No
3,49607,fisica,ramon arturo garcia chavez,amonestacion,NaN,0.0,"lmv - por omitir el auditor externo, al dictam...","LMV - Art. Regulador: 104, fracción III Art. S...",al dictaminar los estados financieros de una e...,2014,2018-05-21,No,No,No,No,Si,2018,No
4,49608,fisica,sergio vargas vargas,amonestacion,NaN,0.0,"lmv - por omitir el auditor externo, al dictam...","LMV - Art. Regulador: 104, fracción III Art. S...",al dictaminar los estados financieros de una e...,2015,2018-05-21,No,No,No,No,Si,2018,No
5,47980,fisica,humberto garza valdez,amonestacion,NaN,0.0,lmv - omitir difundir informacion relevante\n(...,"LMV - Art. Regulador: 44, V Art. Sanción: 392,...",en su caracter de director general omitio difu...,2015,2017-11-16,No,No,No,No,Si,2017,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16142,40782,moral,caja popular juventino rosas,NaN,Sociedades cooperativas de ahorro y préstamo,0.0,"lrascap - omitir presentar, dentro del plazo e...","LRASCAP - Art. Regulador: 34, último párrafo A...",el consejo de vigilancia no realiza las funcio...,2011,2016-09-21,No,No,No,No,Si,2016,No
16143,40822,moral,almacenadora inter americana,NaN,Organizaciones y Actividades Auxiliares del Cr...,0.0,lgoaac - no presentar los documentos o la info...,"LGOAAC - Art. Regulador: 56 Art. Sanción: 89, ...",el importe correspondiente al monto no utiliza...,2012,2016-09-29,No,No,No,No,Si,2016,No
16144,40823,moral,almacenadora inter americana,NaN,Organizaciones y Actividades Auxiliares del Cr...,0.0,lgoaac - registros contables incorrectos (orga...,"LGOAAC - Art. Regulador: 52 Art. Sanción: 89, ...",omision de registro contable y falta de presen...,2012,2016-09-29,No,No,No,No,Si,2016,No
16145,40834,moral,"ci banco, s. a., institucion de banca multiple",NaN,Instituciones de banca múltiple,0.0,lic - prevencion de lavado de dinero,"LIC - Art. Regulador: 115 Art. Sanción: 115, p...",esa sociedad omitio actualizar la informacion ...,2009,2016-09-28,No,No,No,No,Si,2016,Si


In [110]:
data_moral= data[data['tipo_de_persona']== 'moral']


In [111]:
data_moral['clasificacion'] = np.where(data_moral['infractor'].str.contains('hsbc|bbva|banco nacional de mexico|banamex|banorte|banco mercantil|santander', regex=True), 'A' , 'B' )


<ipython-input-111-2c1754cf1fd9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_moral['clasificacion'] = np.where(data_moral['infractor'].str.contains('hsbc|bbva|banco nacional de mexico|banamex|banorte|banco mercantil|santander', regex=True), 'A' , 'B' )


In [112]:
condition= [ data_moral['infractor'].str.contains('hsbc', regex=True),\
            data_moral['infractor'].str.contains('bbva', regex=True),\
           data_moral['infractor'].str.contains('banco nacional de mexico|banamex', regex=True),\
           data_moral['infractor'].str.contains('banco mercantil del norte|banorte', regex=True),\
           data_moral['infractor'].str.contains('santander', regex=True)]
choices= ['HSBC','BBVA', 'Citibanamex', 'Banorte', 'Santander']

In [113]:
data_moral['short_5_grandes'] = np.select(condition,choices, default= 'Otro')

<ipython-input-113-62e350daf78b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_moral['short_5_grandes'] = np.select(condition,choices, default= 'Otro')


In [114]:
data_moral= data_moral[['id_sancion','clasificacion', 'short_5_grandes']]

In [115]:
data_concat = pd.merge(data,data_moral,how='left',
                        on=['id_sancion'])


In [116]:
data_concat.head()

,id_sancion,tipo_de_persona,infractor,tipo_de_sancion,sub_sector,monto,conducta_sancionada,normatividad_infringida,descripcion_de_la_conducta,anio_infraccion,fecha_de_imposicion,pagada,grave,reincidente,resolucion_firme,susceptible_de_impugnacion,anio_imposicion,lavado,clasificacion,short_5_grandes
0,48073,fisica,leon cabrera flores,amonestacion,NaN,0.0,lmv - el sistema automatizado para la recepcio...,"LMV - Art. Regulador: 180 Art. Sanción: 392, f...","el sistema automatizado para la recepcion, reg...",2015,2017-11-30,No,No,No,No,Si,2017,No,NaN,NaN
1,48074,fisica,jaime antonio gonzalez remis,amonestacion,NaN,0.0,lmv - el sistema automatizado para la recepcio...,"LMV - Art. Regulador: 180 Art. Sanción: 392, f...","el sistema automatizado para la recepcion, reg...",2015,2017-11-30,No,No,No,No,Si,2017,No,NaN,NaN
2,49607,fisica,ramon arturo garcia chavez,amonestacion,NaN,0.0,"lmv - por omitir el auditor externo, al dictam...","LMV - Art. Regulador: 104, fracción III Art. S...",al dictaminar los estados financieros de una e...,2014,2018-05-21,No,No,No,No,Si,2018,No,NaN,NaN
3,49608,fisica,sergio vargas vargas,amonestacion,NaN,0.0,"lmv - por omitir el auditor externo, al dictam...","LMV - Art. Regulador: 104, fracción III Art. S...",al dictaminar los estados financieros de una e...,2015,2018-05-21,No,No,No,No,Si,2018,No,NaN,NaN
4,47980,fisica,humberto garza valdez,amonestacion,NaN,0.0,lmv - omitir difundir informacion relevante\n(...,"LMV - Art. Regulador: 44, V Art. Sanción: 392,...",en su caracter de director general omitio difu...,2015,2017-11-16,No,No,No,No,Si,2017,No,NaN,NaN


In [117]:
data_concat['clasificacion'] = data_concat['clasificacion'].fillna('C')
data_concat['short_5_grandes'] = data_concat['short_5_grandes'].fillna('Persona Fisica')

In [118]:
data_concat.loc[(data_concat['monto'] != 0) & (data_concat['tipo_de_sancion'].isnull()),'tipo_de_sancion']= "multa (sancion pecuniaria)"

In [119]:
data_concat.loc[(data_concat['monto']== 0) & (data_concat['tipo_de_sancion'].isnull()),'tipo_de_sancion']='amonestacion'

In [120]:
#data_concat[data_concat['id_sancion']== 42184]

In [121]:
data_concat[data_concat.monto.isnull()]

,id_sancion,tipo_de_persona,infractor,tipo_de_sancion,sub_sector,monto,conducta_sancionada,normatividad_infringida,descripcion_de_la_conducta,anio_infraccion,fecha_de_imposicion,pagada,grave,reincidente,resolucion_firme,susceptible_de_impugnacion,anio_imposicion,lavado,clasificacion,short_5_grandes


In [122]:
data['tipo_de_sancion'].unique().tolist()

['amonestacion',
 'inhabilitacion',
 'suspension',
 'multa (sancion pecuniaria)',
 nan,
 'revocacion',
 'cancelacion del registro']

In [123]:
data_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14947 entries, 0 to 14946
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id_sancion                  14947 non-null  int64         
 1   tipo_de_persona             14947 non-null  object        
 2   infractor                   14947 non-null  object        
 3   tipo_de_sancion             14947 non-null  object        
 4   sub_sector                  14651 non-null  object        
 5   monto                       14947 non-null  float64       
 6   conducta_sancionada         14947 non-null  object        
 7   normatividad_infringida     14947 non-null  object        
 8   descripcion_de_la_conducta  14947 non-null  object        
 9   anio_infraccion             14947 non-null  object        
 10  fecha_de_imposicion         14947 non-null  datetime64[ns]
 11  pagada                      14947 non-null  object    

In [124]:
data_concat.to_excel('cnbv_limpio.xlsx', index=False)

In [44]:
#data_concat.to_csv('cnbv_limpio.csv', index=False)